# COGS 188 - Final Project

# Poker Bot

## Group members

- Zak Bamford
- Sreetama Chowdhury
- Joseph Edmonston

# Abstract 

We seek to create a poker bot that will play optimally in order to provide the best possible returns. After developing our initial bot, we will compare its performance against a bot that takes random actions as a baseline. From there, we will iteratively improve our bot, and compare its performance against prior iterations. The bots' performance will be measured by the amount of money they win while playing against other bots over a large number of games to minimize the effects of variance. To improve the bot, we will use Monte Carlo methods to estimate the values of each hand and each set of table cards. These values will then be used to determine whether folding, calling, or raising would be the best play based on the win probability of the hand. Our bot was successful in outperforming a random bot (~60-70% of games won in varying by iteration) and improving it's play by playing against itself and previous iterations of itself. There are still many avenues for improvement.

# Background

Poker is a popular card game that blends skill, strategy, and chance. Players engage in betting on the strength of their hands throughout multiple rounds, with the goal of either having the best hand or convincing their opponents to fold through strategic bluffing. In essence, poker is a "game of chip management," emphasizing the importance of making decisions based on one's chip count relative to that of their opponents. In addition, poker is a mathematically complex game; there are 2,598,960 unique poker hands and players can choose to bet as much as they want. This makes poker a great candidate for an AI algorithm, as humans could never evaluate all the possible states.

Several aspects of poker can be algorithmized -- the calculation of probabilities, pot odds, and hand strengths to determine the best actions, such as betting, raising, or folding. Algorithms can model and predict opponents' behavior by analyzing historical data, enabling adaptive strategies that counteract different playing styles. Game theory principles can be employed to devise strategies that minimize losses and maximize gains over the long term, even in the face of unpredictable opponents. There are multiple approaches to poker bots; these include basic strategy bots and bots that adapt to their opponents' moves. Basic strategy bots play a mathematically optimal strategy which can be exploited by strong opponents. On the other hand, adaptive bots attempt to change their strategies based on what their opponents do, but this may lead to suboptimal play since bots are not good at reading humans' bluffs.

It's important for us to note that this "problem" isn't quite a novel one -- there are already a range of poker bots available for use, with varying levels of skill and success at playing. CardsChat, a forum and learning community for poker players, explains that while poker bots have been around for a good few years, they're still frowned upon and very much prohibited in online game rooms that play for real money. While bots may be profitable in lower-stakes poker games, high level poker players are generally more psychologically involved in gameplay in a way bots are too mathematically driven to emulate. There are a number of tips provided by various sources to human poker players to help identify poker bots in the real world, such as identical decision timing and repetitive moves. Once found and investigated, bots are generally banned permanently.

Since this problem has been worked on extensively in the past, environments for testing poker bots already exist. One of these environments is the neuron_poker environment by dickreuter on GitHub, which is a poker environment for OpenAI Gym. This environment comes with a variety of existing bots, including a bot that plays random moves, and allows for the development of custom bots. This allows much of the groundwork of developing the game of poker to be skipped, allowing us to focus on developing our bot.

As a mathematical/programming exercise, though, they're interesting! MIT Pokerbots, for example, is an annual one-month-long competition that gives teams of students one month to create and compete with a completely autonomous poker bot, with rewards including both financial prizes and attention from technological and trading companies!

# Problem Statement

In each possible state of a poker game consisting of the bot’s hand, the table cards, and the pot, we will attempt to find the optimal policy that maximizes our rewards from that state while playing against another poker player. Since poker is partially a game of chance, the bot’s profit over a large number of games will be measured. Initially we will design our bot for Heads-up poker, poker with only two players. This may be expanded to multi bot play.

# Data

Since we will have bots play against each other for training and testing, we should not need an external dataset.

data was represented in an 840k sized Q value matrix. Matrix was initially much larger but a reduction in size was necessary to allow for meaningful training in a reasonable time period. Represented in the shape was the predicted equity/probability of winning, 2 player stacks as a fraction of the initial stack, game states, and the action space: fold, call, check, raise, etc. 

```
        shape = (100,10,10, 4, env.action_space.n)
```




# Proposed Solution

We will use a monte carlo training method. We will run simulations, having our bot play against itself as it learns along with bots that play at random.

The number of possible states without abstraction in poker is far too large to be able to train a model to cover all these states. Because of this, our solution will attempt to reduce the state and action space in a manner that still allows us to have an effective bot.

The state space will be reduced to a set of the following:

 the win probability of your hand to a randomly dealt opponent’s hand. 

The actions that have led up to this state from the start of the round.

The bucketed quantities of money both players have. Again to decrease our state space, we think it might be useful to, instead of representing every possible amount of money our players can have, to represent it as a bucketed ratio of the initial buy in. 

For example, the buy in is represented as 1, the max value you can have is 2, the sum opponent value + your value = 2. Then when you have .846… of your initial buy in we would just bucket it to .85.

The current round of betting that we are in ( pre flop, flop, turn, river)

For example a state might look like (55, preflop, opponent-check, , user=.85, opponent =1.15)
and you would pick an action from this state

The set of actions will also be simplified, in particular what our bot might bet . Our bot can either fold, check, or raise. When it decides to raise, instead of considering every possible bet it will draw from a set of possibilities. It can either decide to bet on the current amount in the pot, the amount it currently owns, or the amount the opponent owns. Then it can bet either ¼, ½, ¾, or the entire quantity. So 12 possible betting options, which can be expanded for greater granularity if needed.

The rewards will be assigned at the end of the round, and will go back up the action tree with a decay value. The reward will just be the amount of money won or lost. It may also be interesting to look at what we might have won/lost if we kept playing and use curLost - might have gained as our reward instead. 

Our solution has a reasonable chance of success because even with an extremely large reduction of states from the true number of states The most important key points of information remain. For example, a low probability hand where your opponent has been consistently betting is all you really need to know to know that you should fold. 

Because of the reduction in states It should be possible to train our bot in a reasonable time frame.


# Evaluation Metrics

One method to evaluate our bot is to run our trained bot against a randomly betting bot. We can run our bot for a set number of games and see its returns. We can also design a few other dummy bots for it to run against like an all in bot or a cautious bot to see how it does.

# Results

We initially explored our problem by setting up a simple training loop within a poker environment, excerpted below. Initially we pursued a training method that received a reward after every action, as that's how the environment was set up to work. However, the rewards returned were often unsatisfactory. It is difficult to reward an action prior to seeing the outcome of the game, and the rewards returned by env.step() were not useful for training. Our winrate against a random action bot was essentially 50% at this stage.


```
env = gym.make(env_name, initial_stacks=self.stack, funds_plot=self.funds_plot, render=self.render,
                use_cpp_montecarlo=self.use_cpp_montecarlo)

ourplayer = OurPlayer(env=env,fromFile=True, writeFile=True)

env.add_player(RandomPlayer())
env.add_player(ourplayer)

#equity, stack 1, stack 2, action
shape = (100,10,10, 4, env.action_space.n)
num_episodes = 30000
for i in range(num_episodes):
    curState = env.reset()
    #print(curState)
    while not env.done:
        curaction = ourplayer.getAction(epsilon=.01)
        everything, reward, done, info = env.step(curaction) #initially used this reward value
    ourplayer.gameDone()
```

We shifted to pursuing a Monte Carlo style training method, where at the end of a hand rewards were distributed downwards based on whether the hand was won or lost. This was done by modifying the environment to call a round over function, that could then distribute rewards based on the outcome of the hand. We trained the bot for 1000 games against the random AI. Then we ran the bot against the AI for 3108 games, where it won 1844 of them, a 59.33% win rate. 

```
def roundOver(self, reward):
print("round over\n\n")
q_table = self.qtable
for n,access in enumerate(self.curActions):
    alpha = self.alphas[access]
    gamma = .7**(len(self.curActions)-n)
    weightedReward = (1/alpha)*(gamma)*(reward - q_table[access])
    print(access, Action(access[-1]), alpha, weightedReward)
    q_table[access] += weightedReward
self.curActions = []
```

![image.png](img/first.png)

Figure 1: The bot very quickly decides that it's beneficial to go all in, as is typical of the average game at this point. The bot has quickly learned that when it's equity value is high it will generally get a nice reward by going all in.


### adjusting parameter values to decrease all-in bias

We believed this all-in tendency early may have been caused by a gamma that is too large. Instead of delaying for more optimal rewards it would go all in now, even if it was punished it would happen at the turn, a few actions away. Retraining the model after increasing gamma (decreasing falloff) We achieved an increase of win rate. Training for 1000 games and simulating 1200 we had 797 wins, up to 66%.

### folding rate

Our bot had a significantly low folding rate, even with adjustments to gamma. Our folding rate was about 1/9, or 11% after running 100 games and keeping track of the number of actions that were folds. For reference, a professional's folding rate would be around 60-70%. Shifting to the bot playing against itself, this rate increased to 

### action selection

We had 2 main ways to select an action, either selecting the most rewarded action in the action space 100% of the time or selecting an action randomly with the Q value being the weight/probability of the action. Against the random bot selecting the most rewarded action granted us a marginally higher win rate (~63% vs ~60%).   

### Training the bot against a non-random opponent (itself)

Following training against the bot, we retrained it against itself, to see how it might learn to deal with an opponent who makes somewhat intelligent bets rather than random bets. After training for 1000 games the notable differences are in the folding rate. Running the bot for 2000 hands we saw 379 folds, ~19% fold rate. This is close to double the fold rate that we had against the random bot. Our all-in rate also decreases immensely, and we are more willing to make small bets.

![image.png](img/image2.png)

Figure 2: the bot becomes more willing to bet non-all in amounts but is still perhaps a bit overzealous. Figure made from game after bot plays against itself for 1000 training games as opposed to playing the random bot.

# Discussion

### Interpreting the result

Our bot was able to do fairly well against the random bot, but it was overeager in grabbing quick rewards. For example, against a random bot the optimal strategy would be to wait with many checks/folds until you get a hand with an extremely high win percentage, and then go all in. Our bot was satisfied to go all in with any hand that had a decent win percent chance, because the net reward on that action would be positive. However, this still is a valid strategy to win, it's just a suboptimal one.

As demonstrated by the low folding rate, we had difficulty coming up with a reward structure that correctly rewarded folding. re reward structure is generally focused on giving rewards for making money and giving punishments for losing money. the problem with folding is that it will always lose money, in it never ends up looking like a good action as a result. in theory the punishments for not folding should hit you harder than the punishment for folding but this did not seem to end up being the case. 

in terms of training the bot against itself the results make sense. When your opponent only calls you when it also has a good hand suddenly the threshold to go all in isn't just 'i'll win 51% of the time", it's "my hand is better than a hand my opponent would be willing to call with". In return the fold rate increasing parallels this change, as now we should fold unless were confident our hand beats the opponents raising hand. The reason they didn't adjust more than they did is most likely due to our poor reward structure as discussed above.

ideally we would come with a system that plays out the entire game when you decide to fold and rewards you in the scenarios where you would have lost the game. this would allow us to more accurately model folding as a desireable action rather than one that only punishes you. However,  it was difficult to modify the environment in a way that we could simulate out the game, and this would need to come in a future study

### Limitations

A significant issue we faced was trraining time. To get even an alright quantity of information we needed >1000 games, as each game was 50-150 actions and our actions space was 840,000 after significant reductions. Just to have an expected value of 1 for each position once you'd need roughly 8400 games, and each game took 1-3 seconds to simulate. In practice it was less since most spaces are unused but it's still indicative of needing significant amounts of training for any amount of coverage. Essentially anytime we wanted to train on new parameters or a modified model we'd need to train it for an hour or two just to have alright coverage, significantly limiting the amount of things we could test. Our 1000 game cutoff was probably too small to be really decent, and we'd probably get better results with longer training periods.

We were also limited by the environment. While it was a massive time save to have an already functional poker environment, it was difficult to modify. even just adding a way to figure out if you won the hand was time consuming, as that was not built in. Working within our own poker environment would have given us much more control over how we train our model.

### Ethics & Privacy

Given our present plans we don't think this project has too many serious ethical considerations -- it's a fairly simple bot, and while poker is a game played with real stakes, we're not using physical money. It's not real gambling at this point, and we aren't using any personal data, it's all data generated playing against our own bots, so there aren't privacy concerns. This bot should not be used or uploaded to play against other individuals in real gambling, as that would be clearly unethical and in violation of poker site rules. but as of right now it's fine. 

### Conclusion

Our work will probably not serve as a cornerstone in the field, as the much more complex problem of limitless Texas Hold'em has already been solved and our bot is far from optimal. However, it still managed to learn to slowly make more rational decisions as it trained first against a random bot and then against itself.

# Footnotes/Sources
- https://www.cardschat.com/poker/strategy/bots/
- https://pokerbots.org/
- https://github.com/dickreuter/neuron_poker
